# Test set evaluation of Gender image classification models

## UTKFace dataset

In [1]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
# UTKFace image directories (unsorted and sorted)
TEST_DIR = 'UTKFace/'
TEST_SORT_DIR = 'UTKFace sorted/'

In [3]:
# labels - age, gender, ethnicity
image_paths = []
age_labels = []
gender_labels = []

for filename in os.listdir(TEST_DIR):
    image_path = os.path.join(TEST_DIR, filename)
    temp = filename.split('_')
    age = int(temp[0])
    gender = int(temp[1])
    image_paths.append(image_path)
    age_labels.append(age)
    gender_labels.append(gender)

In [4]:
# Move UTKFace images into unique directories based on their gender label
# Need only to run once to organize
# m_dir = 'UTKFace sorted/male'
# f_dir = 'UTKFace sorted/female'

# import shutil

# for filename in os.listdir(TEST_DIR):
#     image_path = os.path.join(TEST_DIR, filename)
#     temp = filename.split('_')
#     gender = int(temp[1])
#     if gender == 0:                                # Move to Male folder
#         new_path = os.path.join(m_dir, filename)
#         shutil.move(image_path, new_path)
#     elif gender == 1:                              # Move to Female folder
#         new_path = os.path.join(f_dir, filename)
#         shutil.move(image_path, new_path)

In [5]:
# Grayscale test set images
test_ds_gs = tf.keras.utils.image_dataset_from_directory(TEST_SORT_DIR, 
                                                         seed=123, 
                                                         batch_size=128,
                                                         image_size=(224, 224),
                                                         color_mode="grayscale")
# RGB test set images
test_ds_rgb = tf.keras.utils.image_dataset_from_directory(TEST_SORT_DIR, 
                                                         seed=123, 
                                                         batch_size=128,
                                                         image_size=(224, 224),
                                                         color_mode="rgb")

Found 23708 files belonging to 2 classes.
Found 23708 files belonging to 2 classes.


In [6]:
test_labels = np.concatenate([y for x, y in test_ds_gs], axis=0)

In [7]:
test_labels

array([1, 1, 0, ..., 1, 0, 0], dtype=int32)

In [8]:
# Prediction function
def reload_predict(model, dataset):
    # Reload model from saved .h5 files
    model.summary()
    test_ds = dataset
    # make predictions on test set images
    test_predictions = model.predict(test_ds)
    # Evaluate the model
    test_predictions_rounded_labels=np.argmax(test_predictions, axis=1)
    print(classification_report(test_labels, test_predictions_rounded_labels))  

## Evaluate models

### VGG-16 Model 1b-gs: Grayscale + Frozen layers

In [9]:
# Recreate model instance
M1b_gs = keras.models.load_model('GC weights/Frozen/mod1b-gs/mod1b_gs_frozen.h5')

In [10]:
reload_predict(M1b_gs, test_ds_gs)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 3)       30        
                                                                 
 vgg16 (Functional)          (None, 1000)              138357544 
                                                                 
 dense (Dense)               (None, 2)                 2002      
                                                                 
Total params: 138,359,576
Trainable params: 2,032
Non-trainable params: 138,357,544
_________________________________________________________________
186/186 [==============================] - 47s 141ms/step
              precision    recall  f1-score   support

           0       0.48      0.37      0.42     11317
        

### VGG-16 Model 1b-RGB: RGB + Frozen layers

In [11]:
# Recreate model instance
M1b_rgb = keras.models.load_model('GC weights/Frozen/mod1b-RGB/mod1b_RGB_frozen.h5')

ValueError: No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x7f7de0307df0>.

In [ ]:
reload_predict(M1b_rgb, test_ds_rgb)